# This file presents a pipeline to classify the bags

The first step is to do a PCA with the build images from the bags initially to reduce the feature dimensionalty. The second step is an SVM to classify in between the three bags.

In [1]:
import os, cv2
import numpy as np
import sklearn, random
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline 
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, KFold

## Collecting the data of the detected bags

In [2]:
# Getting the paths to all the detected bags
path_to_bags = os.path.join(os.getcwd(), 'data/processed_bags')
bags_to_label = {'mixed_recycling':0, 'general_waste':1, 'green_sack':2}
all_paths = [os.path.join(path_to_bags, form + '/' + index) for form in bags_to_label for index in os.listdir(os.path.join(path_to_bags, form))]

## Utils

In [3]:
def rotation(image, angle):
    '''This function rotates an image, for data augmentation.
    input:
        - image, np.array of the image to be rotated
        - image, np.array of the rotated image'''
    
    # randomly select the angle
    angle = int(random.uniform(-angle, angle))
    h, w = image.shape[:2]
    
    # create the rotation matrix
    T = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    
    # rotate the image
    image = cv2.warpAffine(image, T, (w, h))
    
    return image

In [4]:
def get_cv_iterator(train_val_ind, n_splits=5):
    '''This function creates a cross validation iterator which takes into account the
    difference between the training and validation set. Allows to select the transfomed/augmented
    instance of the training set, and only the untouched examples for the validation set.
    input:
        - train_val_ind, list, indices of the train and validation datasets.
        - n_splits, int, number of folds
    output:
        - tuple(cv), tuple of lists, iterator for the cross validation.'''
    
    # initialise the parameters
    kf = KFold(n_splits)
    cv = []
    
    # add the respective indices for each of the dataset
    for train, test in kf.split(train_val_ind):
        # multiply by 4 because of the three transforms added for each image
        new_train = 4*np.array(train)
        new_test = 4*np.array(test)
        full_train = new_train
        
        # add indices of the transforms for each example of the training set
        for i in range(1, 4):
            full_train = np.append(full_train, new_train+i)
        
        # build the iterator
        cv.append((full_train, new_test))
        
    return tuple(cv)

In [14]:
def create_and_save_transforms(all_paths):
    '''This function creates and saves the augmentations for each image.
    input:
        - all_paths, list of all the paths leading to the images
    output:
        - X_transformed, np.array of the data of the transformed images
        - y_transformed, np.array of the labels of the transformed images'''
    img_data = np.empty((0, 7500))
    img_label = np.empty((0, 7500))
    
    for path in all_paths:
        image = cv2.imread(path)
        img_flat = np.ndarray.flatten(image).reshape((1,-1))
        
        # applying transforms for data augmentation
        img_rot_flat = np.ndarray.flatten(rotation(image, 20)).reshape((1,-1))
        img_flippedh_flat = np.ndarray.flatten(cv2.flip(image, 1)).reshape((1,-1))
        img_flippedv_flat = np.ndarray.flatten(cv2.flip(image, 0)).reshape((1,-1))
        
        # gettinf the data into the arrays
        img_data = np.append(img_data, np.array([img_flat, img_flippedv_flat, img_rot_flat, img_flippedh_flat]).reshape((4, 7500)), axis=0)
        label = bags_to_label[path.split('/')[-2]]
        img_label = np.append(img_label, np.array([label, label, label, label]).reshape((4, 1)))


    np.save('data/tranformed/transformed_bags_data.npy', img_data)
    np.save('data/tranformed/transformed_bags_labels.npy', img_label)

    X_transformed = np.load('data/tranformed/transformed_bags_data.npy')
    y_transformed = np.load('data/tranformed/transformed_bags_labels.npy').reshape(-1, 1)
    
    return X_transformed, y_transformed

In [15]:
def get_transforms(indices, all_paths, train):
    '''Gives the data for the requested images.
    input:
        - indices, list of required indices
        - all_paths, list of the paths to the images
        - train, str, to say if the augmentations are required or not
    output:
        - X_transformed, np.array of the relevant images
        - y_transformed, np.array of the relevant labels'''
    X_transformed = np.load('transformed_bags_data.npy')
    y_transformed = np.load('transformed_bags_labels.npy').reshape(-1, 1)
    if train=='train':
        new = 4*np.array(indices)
        full_indices = new
        for i in range(1, 4):
            full_indices = np.append(full_indices, new+i)
        return X_transformed[full_indices, :], y_transformed[full_indices, :]
    else:
        return X_transformed[4*np.array(indices), :], y_transformed[4*np.array(indices), :]

# Classification Pipeline

### The following section provides the code for a cross validation and hyper-parameter tuning over the images. The first set proves the expected results on a test set when performing the hyper-parameter tuning pipeline.

A PCA is applied on the images to perform dimensionality reduction. These are then used as featrues for an SVM to perform classification.

In [16]:
os.mkdir('data/tranformed/')
create_and_save_transforms(all_paths)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 4., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.]]),
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [

In [8]:
# evaluating the hyper-parameter tuning by providing the score on the test samples
def evaluate_hp_tuning(cv, all_paths):
    '''This fucntion serves to evaluate the perfomance of the hyper parameter tuning 
    on each of the cross validations. 
    input:
        - cv, tuple iterator for the cross validation
        - all_paths, list of all the paths to the untouched images
    output:
        - scores, array of the scores for each cross validation
        - mean, float, mean of the scores
        - var, flaot, variance of the scores'''
    scores = []
    for ind, (train_val_ind, test_ind) in enumerate(cv):
        pca = PCA()
        svm = SVC()
        std1 = StandardScaler()
        std2 = StandardScaler()

        pipe = Pipeline(steps=[('std1', std1), ('pca', pca), ('std2', std2), ('svm', svm)])

        param_grid = {
        'pca__n_components': [5, 15, 30, 45, 60, 75],
        'svm__C': [1, 30, 50, 100, 150],
        'svm__kernel':['rbf', 'poly']
        }
        
        cv_in = get_cv_iterator(train_val_ind, n_splits=10)

        X_train_val, y_train_val = get_transforms(train_val_ind, all_paths, 'train')
        X_test, y_test = get_transforms(test_ind, all_paths, 'test')

        search_svm = GridSearchCV(pipe, param_grid, cv=cv_in, n_jobs=-1)
        search_svm.fit(X_train_val, y_train_val.ravel())

        scores.append(search_svm.score(X_test, y_test.ravel()))
    
    # Average scores obtained on test samples and their variance
    return scores, np.array(scores).mean(), np.array(scores).var()

In [9]:
# creating folds for the full cross validation, shuffling paths because as for now they are ordered per class
random.seed(0)
random.shuffle(all_paths)
cv = KFold(n_splits=10).split(range(len(all_paths)))
results, mean, var = evaluate_hp_tuning(cv, all_paths)
print(mean)

0.64


## Final cross-validation over all test samples - provides hyper-parameters for porduction

In [10]:
def get_hp_production(all_paths):
    '''Perfom final hyperparamter tuning to get the final score and final model for test time.
    input:
        - all_paths, list of all the paths leading to the images
    output:
        - params, dict, parameters of the best model
        - best_score, float, mean best score of the best model over the cross validation
    '''
    X, y = get_transforms(list(range(len(all_paths))), all_paths, 'train')
    cv = get_cv_iterator(range(len(all_paths)), n_splits=5)
    
    pca = PCA()
    svm = SVC()
    std1 = StandardScaler()
    std2 = StandardScaler()
    
    pipe_final = Pipeline(steps=[('std1', std1), ('pca', pca), ('std2', std2), ('svm', svm)])
    
    param_grid = {
    'pca__n_components': [5, 10, 50, 100],
    'svm__C': [1, 10, 30, 100, 150, 200],
    'svm__kernel':['rbf', 'poly']
    }
    
    search_final = GridSearchCV(pipe_final, param_grid, cv=cv, n_jobs=-1)
    search_final.fit(X, y.ravel())
    
    return search_final.best_params_, search_final.best_score_

In [11]:
params, score = get_hp_production(all_paths)

In [12]:
score

0.7982758620689656
